## Feature calculation/assembly - annotate all paralog pairs

Annotate all paralog pairs from Ensembl w/ min. 20% sequence identity (in at least 1 direction)

**Inputs:**
1. Protein complex features (pre-computed)
2. BioGRID PPI features (pre-computed)
3. Ortholog/conservation features (pre-computed)
4. Expression features (pre-computed)
5. Protein age from ProteinHistorian
6. Pfam domains from Ensembl
7. Subcellular location from Protein Atlas

**Output:**
* All paralog pairs annotated with all features

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import os

get_data_path = lambda folders, fname: os.path.normpath(os.environ['3RD_PARTY_DIR']+'/'+'/'.join(folders) +'/'+ fname)
get_local_data_path = lambda folders, fname: os.path.normpath('../../local_data/' +'/'.join(folders) +'/'+ fname)

# Inputs
file_unique_pairs = get_local_data_path(['processed', 'ensembl93'], 'unique_pairs.csv')

file_complex_features = get_local_data_path(['processed','paralog_features'], 'protein_complex_features.csv')
file_ppi_features = get_local_data_path(['processed','paralog_features'], 'ppi_features.csv')
file_ortholog_features = get_local_data_path(['processed','paralog_features'], 'ortholog_features.csv')
file_gtex_expr_feats = get_local_data_path(['processed', 'paralog_features'], 'gtex_expr_features.csv')
file_protein_age = get_data_path(['protein_historian'],'HUMAN_PPODv4_PTHR7-OrthoMCL_wagner1.0_ages_ENS93.txt')
file_pfam_domains = get_data_path(['ensembl', '93'], 'paralog_pfam_domain_ids.txt')
file_protein_location = get_data_path(['protein_atlas'], 'subcellular_location_v19.3.tsv')

# Output
file_all_features = get_local_data_path(['processed', 'paralog_features'], 'all_features.csv')

In [2]:
pairs = pd.read_csv(file_unique_pairs, index_col=0)
paralog_genes = pd.concat([pairs.A1_ensembl, pairs.A2_ensembl]).drop_duplicates()
print('Num pairs:', pairs.shape[0], ', num genes:', paralog_genes.shape[0])
pairs[:1]

Num pairs: 36648 , num genes: 13320


,A1,A2,min_seq_id,max_seq_id,singh_wgd,makino_wgd,WGD,same_chr,closest,family_size,family_id,cds_length_ratio,A1_entrez,A1_ensembl,A2_entrez,A2_ensembl
0,A1BG,OSCAR,0.127273,0.22028,False,False,False,True,False,3,3046,0.578629,1,ENSG00000121410,126014,ENSG00000170909


### 1. Protein complex features
- Either gene in (essential) complex
- Avg. essentiality of complexes in which gene(s) are members

In [3]:
complex_features = pd.read_csv(file_complex_features)
complex_features = pd.merge(pairs[['A1_entrez','A2_entrez']], complex_features)
assert(pairs.shape[0] == complex_features.shape[0])
print('Either in complex:', sum(complex_features.either_in_complex))
print('Avg. complex essentiality: %.2f%%' % (complex_features.mean_complex_essentiality.mean()*100))
complex_features[:1]

Either in complex: 8060
Avg. complex essentiality: 19.86%


,A1_entrez,A2_entrez,either_in_complex,in_same_complex,mean_complex_essentiality,mean_complex_ceres_score,flag_missing_scores,self_complex_only
0,1,126014,False,False,NaN,NaN,False,False


### 2. Protein-protein interactions
- Whether genes in a pair have a protein protein interaction
- Total num interactors
- Fraction of interactors that are shared
- Avg essentiality of shared interactors

In [4]:
ppi_features = pd.read_csv(file_ppi_features)
ppi_features = pd.merge(pairs[['A1_entrez','A2_entrez']], ppi_features)
assert(pairs.shape[0] == ppi_features.shape[0])
print('Interactions between paralog pairs:', sum(ppi_features.interact))
ppi_features[:1]

Interactions between paralog pairs: 2853


,A1_entrez,A2_entrez,interact,n_total_ppi,n_shared_ppi,shared_ppi_jaccard_idx,fet_ppi_overlap,shared_ppi_mean_essentiality,shared_ppi_percent_essential,shared_ppi_mean_ceres_score
0,1,126014,False,22,0,0.0,0.0,NaN,NaN,NaN


### 3. Protein domains
* Num shared domains
* Jaccard index of shared domains

In [5]:
def calculate_shared_pfam_domains(pairs):
    # Load domains file from Ensembl
    domains = pd.read_csv(file_pfam_domains)
    col = 'Pfam domain ID'
    domains = domains.rename(columns={'Gene stable ID':'ensembl_id', col:'pfam_id'}).dropna()
    print('Avg domains per gene:', domains.groupby('ensembl_id').pfam_id.count().mean())
    print('N genes w/ domains: %d = %.2f%%' % (sum(paralog_genes.isin(domains.ensembl_id)), 
                                               sum(paralog_genes.isin(domains.ensembl_id))/paralog_genes.shape[0]*100))

    # Get set of domains for each gene
    domains_per_gene = domains.groupby('ensembl_id').agg({'pfam_id': set}).reset_index()
    display(domains_per_gene[:2])
    
    # Merge set of domains for each gene where applicable (only including cases where both genes have pfam domains)
    df = pd.merge(pairs[['A1_ensembl','A2_ensembl']], domains_per_gene.rename(columns={'ensembl_id':'A1_ensembl'}))
    df = pd.merge(df, domains_per_gene.rename(columns={'ensembl_id':'A2_ensembl'}), on=['A2_ensembl'])

    # Calculate num shared domains and jaccard index (intersection/union)
    df['n_shared_domains'] = df.apply(lambda x: len(x.pfam_id_x.intersection(x.pfam_id_y)), axis=1)
    df['shared_domains'] = df.apply(lambda x: x.n_shared_domains / len(x.pfam_id_x.union(x.pfam_id_y)), axis=1)

    # Merge back with all pairs
    domain_features = pd.merge(pairs[['A1_ensembl','A2_ensembl']], df.drop(columns=['pfam_id_x','pfam_id_y']), how='left')
    domain_features = domain_features.fillna({'n_shared_domains':0, 'shared_domains':0})
    assert(domain_features.shape[0] == pairs.shape[0])
    return domain_features

In [6]:
domain_features = calculate_shared_pfam_domains(pairs)
print('Pairs w/ 1+ shared domain:', sum(domain_features.n_shared_domains > 0))
print('Pairs w/ only shared domains: %.2f%%' % (sum(domain_features.shared_domains== 1)/domain_features.shape[0]*100))
domain_features[:1]

Avg domains per gene: 1.7544378698224852
N genes w/ domains: 12929 = 97.06%


,ensembl_id,pfam_id
0,ENSG00000000003,{PF00335}
1,ENSG00000000005,{PF04089}


Pairs w/ 1+ shared domain: 35476
Pairs w/ only shared domains: 62.69%


,A1_ensembl,A2_ensembl,n_shared_domains,shared_domains
0,ENSG00000121410,ENSG00000170909,1.0,1.0


### 4. Orthologs in S. pombe, S. cerevisiae + cross-species conservation
* Has pombe/cerevisiae ortholog
* Has essential pombe/cerevisiae ortholog
* Conservation score

In [7]:
ortholog_features = pd.read_csv(file_ortholog_features)
ortholog_features = pd.merge(ortholog_features, pairs[['A1','A2']])
assert(ortholog_features.shape[0] == pairs.shape[0])
print('Pairs w/ pombe, cerevisiae, e.coli ortholog:', sum(ortholog_features.has_pombe_ortholog), ',', 
      sum(ortholog_features.has_cerevisiae_ortholog), ',', sum(ortholog_features.has_ecoli_ortholog))
print('Avg. conservation score: %.2f' % (ortholog_features.conservation_score.mean()))
ortholog_features[:1]

Pairs w/ pombe, cerevisiae, e.coli ortholog: 5757 , 8172 , 723
Avg. conservation score: 4.81


,A1,A2,A1_ensembl,A2_ensembl,has_cerevisiae_ortholog,has_essential_cerevisiae_ortholog,has_single_essential_cerevisiae_ortholog,has_cerevisiae_ortholog_ip,has_pombe_ortholog,has_essential_pombe_ortholog,has_pombe_ortholog_ip,has_single_essential_pombe_ortholog,has_ecoli_ortholog,conservation_score
0,A1BG,OSCAR,ENSG00000121410,ENSG00000170909,False,False,False,False,False,False,False,False,False,3


### 5. Protein age from ProteinHistorian
* Mean age for the genes in each paralog pair

In [8]:
def compute_mean_age_for_pair(pairs):
    # Load protein ages, retrieved for all genes where age data was available
    protein_ages = pd.read_csv(file_protein_age, sep='\t', comment='#', names=['protein','age','taxon'])
    print('N genes w/ ages: %d = %.2f%%' % (protein_ages.shape[0], protein_ages.shape[0]/paralog_genes.shape[0]*100))
    display(protein_ages[:1])

    # Extract ensembl id
    protein_ages['ensembl_id'] = protein_ages.protein.apply(lambda x: x.split('~')[0])
    protein_ages = protein_ages[['ensembl_id','age']]
    
    # Assign age for each gene
    paralog_ages = pd.merge(pairs[['A1_ensembl','A2_ensembl']], 
                            protein_ages.rename(columns={'ensembl_id':'A1_ensembl','age':'A1_age'}), how='left')
    paralog_ages = pd.merge(paralog_ages, protein_ages.rename(columns={'ensembl_id':'A2_ensembl','age':'A2_age'}), how='left')
    paralog_ages = paralog_ages.astype({'A1_age':'float', 'A2_age':'float'})
    print('A1-A2 age correlation:', stats.spearmanr(paralog_ages.dropna().A1_age, paralog_ages.dropna().A2_age))
    
    # Calculate mean age
    paralog_ages['mean_age'] = paralog_ages[['A1_age','A2_age']].mean(axis=1)
    print('N. pairs w/ NA age:', paralog_ages[paralog_ages.mean_age.isna()].shape[0])
    print('Avg. age of pairs:', paralog_ages.mean_age.mean())
    
    return paralog_ages[['A1_ensembl','A2_ensembl','mean_age']]

In [9]:
age_features = compute_mean_age_for_pair(pairs)
age_features[:1]

N genes w/ ages: 12625 = 94.78%


,protein,age,taxon
0,ENSG00000092850~Q9UIF3~TEKT2,842.0,Deuterostomia


A1-A2 age correlation: SpearmanrResult(correlation=0.4774473011614666, pvalue=0.0)
N. pairs w/ NA age: 1513
Avg. age of pairs: 591.6427380105307


,A1_ensembl,A2_ensembl,mean_age
0,ENSG00000121410,ENSG00000170909,210.95


### 6. Gene expression features
- Correlation of A1 and A2 gene expression - using all available GTEx expression data
- Min/max mean expression
- Expression asymmetry

In [10]:
gtex_expr_features = pd.read_csv(file_gtex_expr_feats)

# Replace A1/A2 mean expression with min/max mean expression
gtex_expr_features = gtex_expr_features.assign(
    min_mean_expr = gtex_expr_features.apply(lambda x: min(x.A1_mean_expr, x.A2_mean_expr), axis=1),
    max_mean_expr = gtex_expr_features.apply(lambda x: max(x.A1_mean_expr, x.A2_mean_expr), axis=1))
gtex_expr_features = gtex_expr_features.drop(columns=['A1_mean_expr', 'A2_mean_expr'])

display(gtex_expr_features[:1])

,A1_ensembl,A2_ensembl,spearman_corr,pearson_corr,min_mean_expr,max_mean_expr
0,ENSG00000092850,ENSG00000163060,0.457234,0.965852,1.181367,4.524118


In [11]:
# Add prefix for data source
expr_features = gtex_expr_features.set_index(['A1_ensembl','A2_ensembl']).add_prefix('gtex_').reset_index()
expr_features = pd.merge(pairs[['A1_ensembl','A2_ensembl']], expr_features, how='left')
assert(expr_features.shape[0] == pairs.shape[0])
expr_features[:1]

,A1_ensembl,A2_ensembl,gtex_spearman_corr,gtex_pearson_corr,gtex_min_mean_expr,gtex_max_mean_expr
0,ENSG00000121410,ENSG00000170909,0.273243,-0.030284,9.456192,10.980585


### 7. Protein Subcellular Location

Could treat this as binary feature (T if both paralogs have at least one common localization) or do Jaccard (common localizations / union of localizations).

Using all annotations for Main and Additions locations, expect those with reliability=='Uncertain'  
https://www.proteinatlas.org/about/assays+annotation#ifre 


Alternative? https://compartments.jensenlab.org/Downloads

In [12]:
def compute_colocalization(pairs, protein_atlas_location):
    # Filter out uncertain annotations
    location = protein_atlas_location[protein_atlas_location.Reliability != 'Uncertain'].reset_index(drop=True)

    # Stack location mappings for main and additional location
    dfmain = location.set_index('Gene')['Main location'].apply(lambda x: pd.Series(x.split(';') if not pd.isna(x) else x))\
                      .stack().reset_index(level=1, drop=True).reset_index().rename(columns={0:'location'})

    dfadd = location.set_index('Gene')['Additional location'].apply(lambda x: pd.Series(x.split(';') if not pd.isna(x) else x))\
                     .stack().reset_index(level=1, drop=True).reset_index().rename(columns={0:'location'})

    # Combine main and additional locations
    location_map = pd.concat([dfmain, dfadd]).drop_duplicates().sort_values('Gene')
    
    location_map = location_map.groupby('Gene').location.apply(set).reset_index()
    print('N genes w/ location:', location_map.shape[0])
    print('N paralog genes w/ location: %d = %.2f%%' % (sum(paralog_genes.isin(location_map.Gene)), 
                                                        sum(paralog_genes.isin(location_map.Gene))/paralog_genes.shape[0]*100))
    display(location_map[:1])
    
    # Merge in set for A1 and A2
    df = pd.merge(pairs[['A1_ensembl', 'A2_ensembl']], 
                  location_map.rename(columns={'Gene':'A1_ensembl', 'location':'A1_loc'}), how='left')
    df = pd.merge(df, location_map.rename(columns={'Gene':'A2_ensembl', 'location':'A2_loc'}), how='left')
    # Set lack of data as empty set 
    df['A1_loc'] = df['A1_loc'].apply(lambda d: d if not pd.isnull(d) else set())
    df['A2_loc'] = df['A2_loc'].apply(lambda d: d if not pd.isnull(d) else set())
    df['both_have_loc'] = df.apply(lambda x: len(x.A1_loc)>0 and len(x.A2_loc)>0, axis=1)
    
    df['loc_intersection'] = df.apply(lambda x: x.A1_loc.intersection(x.A2_loc), axis=1)
    df['loc_union'] = df.apply(lambda x: x.A1_loc.union(x.A2_loc), axis=1)
    df['colocalisation'] = df.apply(lambda x: len(x.loc_intersection)/len(x.loc_union) if x.both_have_loc else 0, axis=1)
    df['same_subcell_loc'] = df.loc_intersection.apply(lambda x: len(x) > 0)

    location_features = df[['A1_ensembl', 'A2_ensembl', 'colocalisation', 'same_subcell_loc','both_have_loc']]
    return location_features

In [13]:
protein_atlas_location = pd.read_csv(file_protein_location, sep='\t')
display(protein_atlas_location[:1])

,Gene,Gene name,Reliability,Main location,Additional location,Extracellular location,Enhanced,Supported,Approved,Uncertain,Single-cell variation intensity,Single-cell variation spatial,Cell cycle dependency,GO id
0,ENSG00000000003,TSPAN6,Approved,Cell Junctions;Cytosol,Nucleoli fibrillar center,NaN,NaN,NaN,Cell Junctions;Cytosol;Nucleoli fibrillar center,NaN,Cytosol,NaN,NaN,Cell Junctions (GO:0030054);Cytosol (GO:000582...


In [14]:
location_features = compute_colocalization(pairs, protein_atlas_location)
display(location_features[:1])

N genes w/ location: 11723
N paralog genes w/ location: 7757 = 58.24%


,Gene,location
0,ENSG00000000003,"{Cytosol, Nucleoli fibrillar center, Cell Junc..."


,A1_ensembl,A2_ensembl,colocalisation,same_subcell_loc,both_have_loc
0,ENSG00000121410,ENSG00000170909,0.0,False,False


In [15]:
print('N pairs where both have loc: %d = %.2f%%' % (sum(location_features.both_have_loc), 
                                                    sum(location_features.both_have_loc)/location_features.shape[0]*100))
print('When both have 1+ loc:\nIn same loc: %d = %.2f%%' % (sum(location_features.same_subcell_loc), 
                                           sum(location_features.same_subcell_loc)/sum(location_features.both_have_loc)*100))
print('Fraction shared locations:', 
      location_features[location_features.both_have_loc].colocalisation.median())

N pairs where both have loc: 12197 = 33.28%
When both have 1+ loc:
In same loc: 7220 = 59.19%
Fraction shared locations: 0.3333333333333333


### Merge all features together

In [16]:
def impute_missing_values(pairs):
    
    assert(pairs[pairs.flag_missing_scores & (pairs.either_in_complex==False)].shape[0]==0)
    assert(pairs[(~pairs.either_in_complex) & (~pairs.mean_complex_essentiality.isna())].shape[0]==0)
    assert(pairs[(pairs.n_shared_ppi==0)&(~pairs.shared_ppi_mean_essentiality.isna())].shape[0]==0)
    
    # Mark pairs that will have at least 1 imputed value
    pairs = pairs.assign(has_imputed_val = (pairs.flag_missing_scores |
                                            ((pairs.n_shared_ppi>0) & pairs.shared_ppi_mean_essentiality.isna()) |
                                            pairs.gtex_min_mean_expr.isna() | pairs.gtex_spearman_corr.isna() |
                                            pairs.mean_age.isna()))
    print('Pairs w/ 1+ imputed val: %d = %.2f%%' % (sum(pairs.has_imputed_val), sum(pairs.has_imputed_val)/pairs.shape[0]*100))

    # 1. For pairs in protein complexes, fill in feature mean, otherwise fill in 0
    # Feature mean calculated before filling in NAs, so won't include 0s in calculation
    print('Pairs in complex, no complex member scores: %d = %.2f%%' % (
        sum(pairs.flag_missing_scores), sum(pairs.flag_missing_scores)/sum(pairs.either_in_complex)*100))

    for col in ['mean_complex_essentiality','mean_complex_ceres_score']:
        pairs.loc[pairs.flag_missing_scores,:] = pairs.loc[pairs.flag_missing_scores,:].fillna({col:pairs[col].mean()})
        pairs = pairs.fillna({col:0})
    
        
    # 2. For pairs w/ shared PPIs, fill in feature mean, otherwise fill in 0
    print('Pairs w/ shared ppi, no ppi scores: %d = %.2f%%' % (
        sum((pairs.n_shared_ppi>0) & pairs.shared_ppi_mean_essentiality.isna()), 
        sum((pairs.n_shared_ppi>0) & pairs.shared_ppi_mean_essentiality.isna()) / pairs.shape[0]*100))

    for col in ['shared_ppi_mean_essentiality', 'shared_ppi_percent_essential', 'shared_ppi_mean_ceres_score']:
        pairs.loc[(pairs.n_shared_ppi>0),:] = pairs.loc[(pairs.n_shared_ppi>0),:].fillna({col:pairs[col].mean()})
        pairs = pairs.fillna({col:0})

    # 3. Fill NA for expression and gene age
    print('Pairs w/out GTEX mean expr: %d = %.2f%%' % (sum(pairs.gtex_min_mean_expr.isna()), 
                                                       sum(pairs.gtex_min_mean_expr.isna())/pairs.shape[0]*100))
    print('Pairs w/out GTEX expr corr: %d = %.2f%%' % (sum(pairs.gtex_spearman_corr.isna()), 
                                                       sum(pairs.gtex_spearman_corr.isna())/pairs.shape[0]*100))
    print('Pairs w/out gene age: %d = %.2f%%' % (sum(pairs.mean_age.isna()), sum(pairs.mean_age.isna())/pairs.shape[0]*100))
    
    for col in [col for col in pairs.columns if col.startswith('gtex') or col in ['mean_age']]:
        pairs = pairs.fillna({col:pairs[col].mean()})

    return pairs

In [17]:
# Some features were already in the paralog pairs file
data = pairs.assign(sorted_gene_pair = pairs.apply(lambda x: '_'.join(sorted([x.A1, x.A2])), axis=1))
data = data[['sorted_gene_pair','A1','A2','A1_ensembl','A2_ensembl','A1_entrez','A2_entrez', 'closest',
              'min_seq_id', 'max_seq_id', 'WGD', 'family_size','same_chr', 'cds_length_ratio']]
data = data.rename(columns={'min_seq_id':'min_sequence_identity', 'max_seq_id':'max_sequence_identity'})

data = pd.merge(data, complex_features)
data = pd.merge(data, ppi_features)
data = pd.merge(data, domain_features)
data = pd.merge(data, expr_features)
data = pd.merge(data, ortholog_features)
data = pd.merge(data, age_features)
data = pd.merge(data, location_features)

#display(data.isna().any()[data.isna().any()])

data = impute_missing_values(data)
data = data.round(decimals=6)
assert(data.isna().any().sum()==0)
assert(pairs.shape[0]==data.shape[0])
data[:3]

Pairs w/ 1+ imputed val: 2185 = 5.96%
Pairs in complex, no complex member scores: 60 = 0.74%
Pairs w/ shared ppi, no ppi scores: 477 = 1.30%
Pairs w/out GTEX mean expr: 85 = 0.23%
Pairs w/out GTEX expr corr: 507 = 1.38%
Pairs w/out gene age: 1513 = 4.13%


,sorted_gene_pair,A1,A2,A1_ensembl,A2_ensembl,A1_entrez,A2_entrez,closest,min_sequence_identity,max_sequence_identity,...,has_essential_pombe_ortholog,has_pombe_ortholog_ip,has_single_essential_pombe_ortholog,has_ecoli_ortholog,conservation_score,mean_age,colocalisation,same_subcell_loc,both_have_loc,has_imputed_val
0,A1BG_OSCAR,A1BG,OSCAR,ENSG00000121410,ENSG00000170909,1,126014,False,0.127273,0.220280,...,False,False,False,False,3,210.95,0.0,False,False,False
1,A1BG_TARM1,A1BG,TARM1,ENSG00000121410,ENSG00000248385,1,441864,False,0.149495,0.265233,...,False,False,False,False,3,97.40,0.0,False,False,False
2,OSCAR_TARM1,OSCAR,TARM1,ENSG00000170909,ENSG00000248385,126014,441864,True,0.269231,0.275986,...,False,False,False,False,3,324.50,0.0,False,False,False


In [18]:
data.to_csv(file_all_features, index=0)